# Использование модели метчинга для поиска топ-5 услуг для новых услуг клиник

In [1]:
import pandas as pd
import numpy as np

from matcher.matcher import SentenceSimilarity
from configs.config_matcher import MatcherConfig
from utils.compute_metrics import mark, mapk, get_pivot_table
from tqdm import tqdm

tqdm.pandas()

/home/evgen/desktop/ranker/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-09-19 17:48:12.619738: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-19 17:48:12.627792: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 17:48:12.636404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has 

In [2]:
data_matcher = pd.read_csv("data/data_for_matcher.csv")
data_valid = pd.read_csv("data/data.csv")

In [3]:
data_matcher.sample(5, random_state=42)

,local_name,local_id,parent_id,type,gt_type_name,parent_id_name,site_active,service_name,lpu_name
3635,Групповое занятие лечебной физкультурой при не...,9283.0,NaN,NaN,NaN,NaN,NaN,Групповое занятие лечебной физкультурой при не...,Медси
149,"Катехоламины (адреналин, норадреналин, дофамин...",6064.0,64.0,laboratory_tests,Биохимия,Лабораторная диагностика,True,"Катехоламины в моче (адреналин, норадреналин, ...",Инвитро
2510,"Прием педиатром ребенка повторный, на дому в п...",60.0,NaN,NaN,NaN,NaN,NaN,"Прием педиатром ребенка повторный, на дому в п...",Budu СПб
2506,Нейтрализующие антитела к препаратам интерферо...,6702.0,64.0,laboratory_tests,Инфекции,Лабораторная диагностика,True,Нейтрализующие антитела к препаратам интерферо...,Budu СПб
3204,Холтеровское мониторирование сердечного ритма ...,13571.0,55.0,instrumental_tests,Электрофизиологические методы,Инструментальная диагностика,True,Холтеровское мониторирование сердечного ритма ...,UNIКлиник


In [4]:
config = MatcherConfig()
model = SentenceSimilarity(config=config)

Параметры для конфига указаны в файле `configs/config_matcher.py`

In [5]:
data_valid['preds_top_k'] = data_valid['service_name'].progress_apply(model.get_top_k)
data_valid['preds_local_names_top_k'] = data_valid['preds_top_k'].apply(lambda x: [i['local_name'] for i in x])
data_valid.sample(2, random_state=42)

  0%|          | 0/23700 [00:00<?, ?it/s]/home/evgen/desktop/ranker/matcher/matcher.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.embedder.encode(sentences=texts, convert_to_tensor=True)).clone().detach()
100%|██████████| 23700/23700 [02:52<00:00, 137.02it/s]


,local_name,local_id,parent_id,type,gt_type_name,parent_id_name,site_active,service_name,lpu_name,preds_top_k,preds_local_names_top_k
7944,Протеин C,5549.0,64.0,laboratory_tests,Биохимия,Лабораторная диагностика,True,Протеин С,"ООО ""Клинический госпиталь на Яузе""","[{'local_name': 'Протеин C', 'score': 0.851402...","[Протеин C, Протеин S свободный, Витамин С (ас..."
7539,"Прием (осмотр, консультация) врача-оторинолари...",18099.0,NaN,NaN,NaN,NaN,NaN,"Прием (осмотр, консультация) ведущего врача-от...","ООО ""Клинический госпиталь на Яузе""",[{'local_name': 'Осмотр (консультация) врачом-...,[Осмотр (консультация) врачом-радиологом повто...


In [6]:
actual_list = data_valid['local_name'].apply(lambda x: [x]).values.tolist()
predict_list = data_valid['preds_local_names_top_k'].values.tolist()

In [7]:
get_pivot_table(actual_list, predict_list, top_k=5)

,k,mark,mapk,f1_score
0,1,0.821561,0.821561,0.821561
1,2,0.859958,0.859958,0.859958
2,3,0.869100,0.869100,0.869100
3,4,0.873193,0.873193,0.873193
4,5,0.874813,0.874813,0.874813


Пример почему метрики получаются одинаковые

In [8]:
actual = [["a"], ["b"], ["c"]]
predicted = [["b", "a", "c"], ["b", "c", "a"], ["a", "b", "c"]]

In [9]:
mapk(actual, predicted)

0.611111111111111

In [10]:
mark(actual, predicted)

0.611111111111111

In [11]:
pairs = []

for value in tqdm(data_valid['service_name']):
    pairs.append(data_valid[data_valid['service_name'] == value]['local_name'].nunique())
    
pd.Series(pairs).value_counts()

100%|██████████| 23700/23700 [00:16<00:00, 1401.92it/s]


1    23682
2       18
dtype: int64

Существует всего 18 где больше одного метча

In [12]:
def get_error_review(row):
    print(f"Service name в клинике {row['lpu_name']}: {row['service_name']}")
    print(f"True match: {row['local_name']}")
    print(f"Top-k predicted local names: {row['preds_local_names_top_k']}")
    print("-" * 30)
    
for idx, row in data_valid.sample(10, random_state=69).iterrows():
    get_error_review(row)

Service name в клинике Здоровое поколение: Мочевина
True match: Мочевина в сыворотке
Top-k predicted local names: ['Мочевина в сыворотке', 'Мочевина в суточной моче', 'Мочевая кислота в сыворотке', 'Медь в моче', 'Свинец в моче']
------------------------------
Service name в клинике Инвитро: Cyfra-21-1 (Фрагмент Цитокератина 19)
True match: Фрагменты цитокератина 19 CYFRA 21-1
Top-k predicted local names: ['Фрагменты цитокератина 19 CYFRA 21-1', 'Цитохром P450 2C19. Генотипирование по маркеру CYP2C19 (G681A; *2)', 'Цитохром P450 2C19. Генотипирование по маркеру CYP2C19 (C(-806)T;*17)', 'ПЦР анализ химерного гена RUNX1/RUNX1T1 t(8;21)', 'ПЦР анализ химерного гена E2A/PBX1\xa0 t(1;19)']
------------------------------
Service name в клинике ООО «МЕДИЦИНСКИЙ ЦЕНТР АТЛАС»: Иммуноцитохимическое исследование 5 маркеров
True match: Иммуноцитохимическое исследование материала (5 маркеров) (кроме PTEN)
Top-k predicted local names: ['Иммуноцитохимическое исследование материала (5 маркеров) (кроме

In [13]:
model.get_top_k("Антитела класса IgG к вирусу Varicella-Zoster", top_k=5)

/home/evgen/desktop/ranker/matcher/matcher.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.embedder.encode(sentences=texts, convert_to_tensor=True)).clone().detach()


[{'local_name': 'Varicella Zoster Virus, IgG', 'score': 0.8591366},
 {'local_name': 'Varicella Zoster Virus, IgA', 'score': 0.7878473},
 {'local_name': 'Varicella Zoster Virus, IgM', 'score': 0.771694},
 {'local_name': 'Антитела к нуклеосомам класса IgG', 'score': 0.76468045},
 {'local_name': 'Антитела к Poliomyelitis virus, IgG', 'score': 0.70515966}]

# Итоги
### Описание подхода
1. Для решения задачи `Поиск топ-5 эталонных услуг для новых услуг клиник` использовался метод решения, который сравнивает эмббединг входного текста с эмббедингами локальных услуг с помощью косинусного расстояния и выводит `top_k = 5` похожих.
2. Необходимо было выбрать какой эмббедер использовать, воспользовался бенчмарком encodechka [репозиторий](`https://github.com/avidale/encodechka`]) и выбрал лучшую модель, которая может решать задачу `STS - Semantic text similarity`, по информации с ресурса encodechka, самой подходящей моделью оказалась `deepvk/USER-bge-m3`.

### Анализ результатов
1. Был проведен расчёт метрик `mapk` - mean average precision k, `mark` - mean average recall k и `f1-score` для разных вариаций K и результат расчёта метрик выведен в таблицу.
2. С уверенностью можно сказать, что алгоритм работает вполне хорошо, например для примера `Антитела класса IgG к вирусу Varicella-Zoster`, получились хорошие результаты и высокий `score`
3. Все метрики имеют одинаковые значения ввиду того, что существует очень малое количество семплов в котором `service_name` соотетствует нескольким `local_name`. Так как у нас всего 18 таких пар где значение больше 1.

### Возможные улучшения 
1. Данную задачу можно решить с помощью `Sentence Pair Classification`
2. Провести более глубокий анализ датасета, корректно заполнить пропуски эвристиками и проконсультироваться с специалистом. Использовать другие столбцы как доп.фичи
3. Рассчитать метрики такие как : `MRR`, `NDCG`
4. Разработать сервис с помощью `API/DOCKER`